In [1]:
import pandas as pd
import numpy as np

In [2]:
selloutData =pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [4]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"]<0 ,0, selloutData["QTY"])

In [5]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


#### 53주차 제거

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [7]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

### yearValue는 YEARWEEK에서 YEAR만 분리한 값(함수 만들어서 분리하기)

##### 함수 만들기

In [8]:
def customFuntion(inputColumn):
    inputColumn =201601
    yearValue =inputColumn/100
    yearValue = int(yearValue)
    return yearValue

##### YEAR 컬럼 만들고 위 함수 적용하기

In [9]:
selloutData["YEAR"] =selloutData["YEARWEEK"].apply(customFuntion)

In [10]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2016
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2016
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2016
3,A60,PRODUCT47,201402,0.0,0.0,2,2016
4,A60,PRODUCT56,201402,23.0,23.0,2,2016


##### 1) round함수 /가 아니고 // 하면 소숫점 아래자리가 반올림되어 정수형으로 변화 ( 근데 round함수는 반올림을 하기 때문에 50주차 이상이되면 반올림이 되어 연수가 올라가기 때문에 적합하지 않은 방법임)

In [11]:
selloutData["YEAR"] = round(selloutData["YEARWEEK"]//100)

In [12]:
refinedData = selloutData[selloutData.WEEK < 53]

In [13]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2014
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2014
3,A60,PRODUCT47,201402,0.0,0.0,2,2014
4,A60,PRODUCT56,201402,23.0,23.0,2,2014


##### 2 )따라서 int형으로 형 변환을 해주는 게 적합한 방법임


In [14]:
##refinedData = selloutData[selloutData.YEARWEEK%100 < 53]

In [15]:
##refinedData["TEST"] = ((selloutData["YEARWEEK"]/100).astype(int))

In [16]:
##refinedData.head()

###  깰끔~ 해보이기 위해서 정렬!

In [17]:
sortKey = ["REGIONID","PRODUCT", "YEARWEEK"]

In [18]:
sortedData = refinedData.sort_values(sortKey)

In [19]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


##### 여기서 앞 줄 인덱스가 걸리적 거린다면 

In [20]:
step1Data =sortedData.reset_index()

In [21]:
step1Data.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


##### 또는 ..

In [22]:
step1Data =sortedData.reset_index(drop=True)

In [23]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,A00,PRODUCT34,201405,516.0,516.0,5,2014


In [24]:
step1Data =sortedData.reset_index(drop=False)
step1Data.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


### rolling 함수 사용하기!! (정렬을 하고 사용해야 함) 

### 1.기본 이동평균 함수

In [25]:
step1Data["MA"]=step1Data.NEW_QTY.rolling(window=15, center=True).mean()

In [26]:
step1Data.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014,NaN
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014,NaN
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014,NaN
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014,NaN
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014,NaN


#### 위에서 NAN을 없애려면 어떻게 해야 할까? 

##### min_period 이용--rolling함수:이동평균 구하는 함수, 
##### 해석 : (window=15:총 15개의 이동평균을 구하겠다, center=true: 내 자신을 7번째로, 즉 가운데로 놓고 이동평균을 구하겠다, min_periods =1 : 원래는 평균이 15개의 값을 구해야 하지만 최소로 1개 있을 때도 평균을 구하겠다, .mean()는 평균을 구하겠다-> .max(), .min()등으로 사용에 따라 변경 가능 )

In [27]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(
    window=15, center=True, min_periods =1).mean()

In [28]:
step1Data.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


#### result라는 csv 파일로 ./ 현재 경로에 저장하는 명령어

In [29]:
step1Data.to_csv("./result.csv")

In [30]:
pwd

'C:\\Users\\SMART-12\\Python_Fintech\\4.04 Project Seasonality'

#### regionid가 A00이고 PRODUCT가 PRODUCT34인 데이터를 그룹으로 묶어서 Groupkey를 기준으로 new_qty의 평균을 나타내겠다

In [31]:
filteredData = step1Data[(step1Data.REGIONID == "A00")&
                        (step1Data.PRODUCT=="PRODUCT34")]

In [32]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [33]:
selectedData=filteredData.groupby(groupKey)[["NEW_QTY"]].mean()

In [34]:
selectedData.columns=["QTY_NEW"]

In [35]:
selectedData.head()

QTY_NEW
REGIONID PRODUCT   YEAR            
A00      PRODUCT34 2014  275.961538
                   2015   86.634615
                   2016   36.576923

데이터 합치기

In [36]:
step1Data.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


### selectedData를 다시 인덱스 형식으로 바꿔준다 (데이터를 조인 시키려면 인덱스 형식이어야 함)

In [37]:
spiltedData=selectedData.reset_index()
spiltedData.head()

,REGIONID,PRODUCT,YEAR,QTY_NEW
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923


### 데이터 조인

In [38]:
mergeKey = ["REGIONID", "PRODUCT", "YEAR"]

In [67]:
mergedData= pd.merge(step1Data, spiltedData,\
                     left_on=mergeKey, right_on = mergeKey, \
                     how = "left")
mergedData.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538


### 데이터가 3배 늘어난다 ???

In [64]:
## mergeKey2 = ["REGIONID", "PRODUCT"]

In [65]:
## mergedData2 = pd.merge(step1Data, spiltedData,\
##                     left_on = mergeKey2, right_on = mergeKey2,\
##                     how = "left")
## mergedData2.head()

In [42]:
### ???

In [43]:
mergedData.to_csv("./mergedData.csv")

In [44]:
groupKey = ["REGIONID", "PRODUCT"]

In [45]:
mergedData.groupby(groupKey)

그룹바이함수 만들기

In [46]:
groupDataSet =mergedData.groupby(groupKey)

In [48]:
len(list(groupDataSet.groups))

794

groupDataSet아래 onegroup을 왜할까? 
롤링함수를 써서 seasonality를 단순하게 구하고 싶음.

모든 데이터를 롤링시키면 추세선이 이상해질 수 있다. 그러니까, 어떤 지역에 대한 추세선을 만들고 싶으면 해당 그룹만의 롤링을 구하는 것이 좋음(롤링이란 이동평균 구하는거)

In [60]:
##하나의 그룹(onegroup)을 가지고 올건데 groupDataSet들의 그룹에서 list세워서 1개만 가지고 올거다!
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[1])

In [58]:
oneGroup.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
156,296,A00,PRODUCT58,201401,0.0,0.0,1,2014,12.600000,NaN
157,1755,A00,PRODUCT58,201402,0.0,0.0,2,2014,11.666667,NaN
158,3123,A00,PRODUCT58,201403,0.0,0.0,3,2014,11.066667,NaN
159,203,A00,PRODUCT58,201404,0.0,0.0,4,2014,9.933333,NaN
160,4367,A00,PRODUCT58,201405,0.0,0.0,5,2014,8.666667,NaN


In [63]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])
oneGroup

,index,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,QTY_NEW
0,298,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,1757,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,3125,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,205,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,4369,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538
5,4313,A00,PRODUCT34,201406,423.0,423.0,6,2014,480.230769,275.961538
6,6288,A00,PRODUCT34,201407,407.0,407.0,7,2014,468.214286,275.961538
7,6403,A00,PRODUCT34,201408,364.0,364.0,8,2014,456.666667,275.961538
8,5158,A00,PRODUCT34,201409,470.0,470.0,9,2014,431.533333,275.961538
9,7437,A00,PRODUCT34,201410,538.0,538.0,10,2014,402.066667,275.961538
